In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd '/content/drive/MyDrive/CS224U'

In [ ]:
!ls

drive  sample_data


In [ ]:

import pandas as pd
import numpy as np
file_path = '/content/drive/MyDrive/CS224U/kaggle_urbandict.csv'
df = pd.read_csv(file_path, error_bad_lines=False)

In [ ]:
len(df)

2580586

In [ ]:
df.head(10)

In [ ]:
filtered_df = df[df['up_votes'] > df['down_votes'] + 10]
filtered_df = filtered_df.dropna(subset=['word', 'definition'])
filtered_df['usage'] = None
len(filtered_df)

In [ ]:
filtered_df['usage'].astype(object)
filtered_df.head(10)

In [ ]:
import requests
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def clean_UD_examples(word):
  response = requests.get(f"http://api.urbandictionary.com/v0/define?term={word}")
  data = response.json()
  examples = []
  if 'list' in data and len(data['list']) > 0:
      definitions = data['list']
      top_definition = max(definitions, key=lambda x: x['thumbs_up'])
      example = top_definition['example']
      cleaned_example = re.sub(r'\[', '', example)  # Remove opening brackets
      cleaned_example = re.sub(r'\]', '', cleaned_example)  # Remove closing brackets
      cleaned_example = re.sub(r'[\n\r]', ' ', cleaned_example)  # Remove line breaks and carriage returns
      word_list = word_tokenize(cleaned_example) # nltk tokenizer after cleaning
      examples.extend(word_list)
      return examples
  return examples


In [ ]:
word = 'queue'
examples = clean_UD_examples(word)
print(examples)

NameError: ignored

In [ ]:
import itertools
for index, row in itertools.islice(filtered_df.iterrows(), 250000):
  word = row['word']
  examples = clean_UD_examples(word)
  filtered_df.at[index, 'usage'] = examples
  print(index)

In [ ]:
len(filtered_df)

625243

In [ ]:
final_df = filtered_df.dropna(subset=['usage'])

In [ ]:
final_df = final_df[final_df['usage'].apply(lambda x: len(x) >0)]
final_df['usage'] = final_df['usage'].apply(lambda x: [word.lower() for word in x])
final_df['word'] = final_df['word'].astype(str).str.lower()

In [ ]:
final_df = final_df.drop(['author'], axis=1)

In [ ]:
final_df['word'] = final_df['word'].apply(nltk.word_tokenize)

In [ ]:
final_df.head(10)

,word_id,word,up_votes,down_votes,definition,usage
0,7,[janky],296,255,Undesirable; less-than optimum.,"[this, janky, shirt, she, gave, me, is, fallin..."
3,12,[hard-core],162,96,anything out of our league that can be good or...,"[i, ca, n't, believe, he, went, to, a, strip, ..."
8,17,[wtf],183,99,what the fuck? ;; use it in place of expletive...,"[wtf, ?, whoth, ?, whentf, ?, wts, ?]"
9,19,[hazy],272,184,A guys state of mind after he sees the girl of...,"[fuckin, hazy, again, !, !, !, !, !, !, !, !, ..."
16,32,[ducket],481,272,a one dollar bill. $1. ;; equivalent to one hu...,"[fucking, ducket, again, ..]"
19,38,[a-hole],68,35,Ass hole,"[los, angeles, lakers, are, a, bunch, of, lazy..."
23,42,[clap],1395,516,a case of gonorrhea,"[i, think, i, got, a, case, of, the, claps, fr..."
28,47,[puke],138,109,to vomit,"[this, crappy, movie, made, johnny, want, to, ..."
29,48,[folks],79,59,"p. noun: People, not necessarily related, to w...","[so, are, your, folks, coming, over, for, east..."
31,50,[dog],1303,915,"n. friend of the same sex, usually male. Deriv...","[``, definition, of, a, dog, :, not, a, cat, '..."


In [ ]:
copy = final_df.copy()
copy = copy[copy.apply(lambda row: row['word'][0] in row['usage'], axis=1)]

In [ ]:
copy['tags'] = None

In [ ]:
copy.head(10)

,word_id,word,up_votes,down_votes,definition,usage,tags
0,7,[janky],296,255,Undesirable; less-than optimum.,"[this, janky, shirt, she, gave, me, is, fallin...",None
8,17,[wtf],183,99,what the fuck? ;; use it in place of expletive...,"[wtf, ?, whoth, ?, whentf, ?, wts, ?]",None
9,19,[hazy],272,184,A guys state of mind after he sees the girl of...,"[fuckin, hazy, again, !, !, !, !, !, !, !, !, ...",None
16,32,[ducket],481,272,a one dollar bill. $1. ;; equivalent to one hu...,"[fucking, ducket, again, ..]",None
28,47,[puke],138,109,to vomit,"[this, crappy, movie, made, johnny, want, to, ...",None
29,48,[folks],79,59,"p. noun: People, not necessarily related, to w...","[so, are, your, folks, coming, over, for, east...",None
31,50,[dog],1303,915,"n. friend of the same sex, usually male. Deriv...","[``, definition, of, a, dog, :, not, a, cat, '...",None
33,54,[raunchy],230,195,"distasteful, obscene, and or just plain gross","[that, was, a, very, raunchy, movie, .]",None
34,55,[energy],127,52,"can be converted from one form to another, but...","[i, so, munch, energy, i, can, go, all, night]",None
36,61,[rental],20,9,means of transportaion that damage is totally ...,"[my, mom, could, n't, afford, to, buy, an, air...",None


In [ ]:
for index, row in copy.iterrows():
    slang = row['word']
    usage = row['usage']
    
    tags = []
    i = 0
    j = 0
    
    while i < len(usage):
        if j < len(slang) and usage[i] == slang[j]:
            tags.append('B')
            j += 1
            # Check for consecutive matching words in column1
            while j < len(slang) and i + j < len(usage) and usage[i + j] == slang[j]:
                tags.append('I')
                j += 1
        else:
            tags.append('O')
        i += 1
    
    copy.at[index, 'tags'] = tags

In [ ]:
len(copy)

75814

In [ ]:
copy.to_csv('/content/drive/MyDrive/final_file.csv', index=False)